In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window,WindowSpec
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
spark = SparkSession.builder.appName('temp').getOrCreate()

In [3]:
df = spark.read.format('csv').load('/FileStore/tables/Machine_learning/data_science_screening_exercise_orders.csv', inferschema = True, header = True)

In [4]:
display(df)

In [5]:
data = df.withColumn('week', date_format(to_date("date", "dd/MMM/yyyy"), "W"))
#data.show()
display(data)

In [6]:
d1[45][0]

In [7]:
d2 = pd.DataFrame({'week':week})
#d2
d3 = sqlContext.createDataFrame(d2)
display(d3)

In [8]:
ordered_data = df.orderBy('customer_id')
count_data = ordered_data.groupBy('customer_id').count()
count_data.show()

In [9]:
ranked_data = ordered_data.withColumn("rank",rank().over(Window.partitionBy("customer_id").orderBy(("date"))))
ranked_data.show()

In [10]:
recent_orders = ranked_data.filter("rank==1")
recent_orders.show()

In [11]:
#data = recent_orders.join(count_data,recent_orders.customer_id == count_data.customer_id)
data = recent_orders.join(count_data,on = 'customer_id')
#data1 = data.withColumnRenamed('count', 'order_count').show()
#data2 = data.drop('value','rank').show()
#data2.show()
data.show()

In [12]:
#data1 = data.withColumnRenamed('count', 'order_count')
data2 = data1.drop('value','rank')
data2.show()

In [13]:
grouped_data = data2.withColumn('day', dayofmonth('date'))
display(grouped_data)

In [14]:
#final_data = grouped_data.withColumn("WEEK", when(grouped_data["week"] == 52,1).otherwise(grouped_data["week"]))
#final_data.withColumn("WEEK",
  #when(col("week")==52, 1).otherwise("week"))
#final_data.printSchema().show()
final1 = final_data.select(final_data.customer_id,
                            final_data.gender,
                            final_data.date,
                            final_data.order_count.cast("integer"),
                            final_data.WEEK.cast("integer")
                             )
final1.printSchema()

In [15]:
final_data.groupBy('order_count').count().show()

In [16]:
display(final1)

In [17]:
#final2 = final1.select('gender', 'order_count', 'WEEK')
#x = final2.select('WEEK').collect()
week_list = final2.select("WEEK").rdd.flatMap(lambda x: x).collect()
order_count_list = final2.select("order_count").rdd.flatMap(lambda x: x).collect()


In [18]:
data_schema =[StructField("Week_list", IntegerType() , True),
         StructField("order_count", IntegerType(), True)]
final_struct = StructType(fields=data_schema)

In [19]:
#ids = [x for x in range(len(week_list))]
s1 = pd.Series(week_list, name = 'week')
s2 = pd.Series(order_count_list, name='weekly_orders')
s3 = pd.concat([s1,s2], axis =1)

In [20]:
df = spark.read.format('csv').load('/FileStore/tables/Machine_learning/final.csv', inferschema = True, header = True)

In [21]:
df1 = df.orderBy('WEEK').select('WEEK', 'gender' , 'order_count')

display(df1)
